<a href="https://colab.research.google.com/github/dyoni2/jiyeon/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

## 검증 세트

모델을 만들고 나서 테스트 세트는 딱 한 번 사용되는 것이 좋음

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [5]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


훈견 세트와 검증 세트의 평가 점수를 이용하여 최적화

최적화 완료 후 테스트 세트로 모델을 최종 평가

In [6]:
print(dt.score(test_input, test_target))

0.8569230769230769


**검증 세트의 문제점**

검증 세트로 인해 훈련 세트가 줄어듬

보통 많은 데이터를 훈련에 사용할수록 좋은 모델

검증 세트를 너무 조금 떼어 놓으면 검증 점수가 불안정함

이러한 문제를 보완할 방법 필요

**교차검증**

검증 세트를 떼어 내어 평가하는 과정을 여러번 반복함

In [11]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01496124, 0.01026368, 0.01185083, 0.01072598, 0.01031375]), 'score_time': array([0.00141788, 0.00138521, 0.00131392, 0.00133944, 0.00129962]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [12]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [23]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [24]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


**그리드 서치**

하이퍼파라미터 탐색과 교차 검증을 한 먼에 수행

즉, cross_validate()함수를 호출할 필요가 없음

In [18]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
print(gs)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})


In [25]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [26]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [27]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [28]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [30]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [31]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [33]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [34]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


**랜덤 서치**

너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있음

**싸이파이**

파이썬의 핵심 과학 라이브러리 중 하나

수치 계산 전용 라이브러리

stats 서브 패키지 사용

통계학과 관련된 패키지

uniform과 randint 클래스

주어진 범위에서 고르게 값을 찾음

In [22]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([0, 2, 9, 0, 2, 8, 9, 4, 4, 4])

10번의 샘플링을 했기 떄문에 고르게 샘플링되는 것 같아 보이지 않음

그러나 샘플링 숫자를 늘리면 쉽게 확인 가능

1000번 샘플링해서 각 숫자의 개수를 확인

In [35]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([110,  91,  92,  97, 113,  95,  97, 115,  82, 108]))

uniform의 사용법도 동일

In [36]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.73748188, 0.44955134, 0.55614729, 0.09814496, 0.00988165,
       0.43734349, 0.50807819, 0.23881963, 0.45145991, 0.09019177])

**예제: 결정 트리 모델에서 min_impurity_decrease, max_depth, min_samples_split, min_samples_leaf 최적값 찾기**

min_samples_leaf: 리프 노드가 되기 위한 최소 샘플의 수

어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작으면 분할 안 됨

샘플링 횟수는 n_iter 매개변수에 지정

In [37]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [46]:
from sklearn.model_selection import RandomizedSearchCV
dt = DecisionTreeClassifier(random_state=42)
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79b94297baf0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x79b942aac610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79b942979420>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79b94353c6a0>},
                   random_state=42)

In [39]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [40]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


최적의 모델은 이미 전체 훈련 세트(train_input, train_target)로 훈련되어 best_estimator_에 저장됨

In [42]:
dt = gs.best_estimator_
print(dt)
print(dt.score(test_input, test_target))

DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=0.00034102546602601173,
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)
0.86


**주의**

교차검증, 그리드 서치, 랜덤 서치는 모델을 훈련하는 것이 아닌

In [43]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79b94297baf0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x79b942aac610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79b942979420>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79b94353c6a0>},
                   random_state=42)

In [44]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
